# GTI771 - Apprentissage machine avancé
## Département de génie logiciel et des technologies de l’information (LogTI)

## Laboratoire 1 - Préparation des données
#### <font color=black> Version 1 - Janvier 2025 - Chargé de lab. Arthur Josi </font>

Les laboratoires sont à faire par groupe de deux ou trois étudiants. Favorisez les groupes de trois.

| NOMS                  | CODE PERMANENT                                   |
|-----------------------|--------------------------------------------------|
| Étudiant1             | Code1                                            |
| Zacharie Morin        | MORZ63310201                                     |
| Raphael Roumat        | ROUR79040002                                     |

# Introduction

Ce premier laboratoire porte sur la préparation de données pour l'apprentissage machine, et l'application d'approche simple de classification. Le problème qui vous est présenté est le problème de fraude bancaire [Bank Account Fraud (NeurIPS 2022)](https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022), dont le but est de detecter les fraudes !

Le dataset a été légèrement modifier pour atteindre les objectifs des différents laboratoire et rendre l'application d'algorithme plus rapide.

Veuillez noter que les données qui vous sont fournies ne sont pas directement adaptées à leur analyse. Une approche naïve où l'on appliquerait un algorithme d'apprentissage machine sans analyse des données au préalable est toujours à éviter, et ceci inclus les laboratoires de ce cours. Au cours de ce premier laboratoire, nous allons donc explorer les données, les préparer, et appliquer une méthode de "template matching" pour déterminer si une transaction est une fraude ou non.


Au cours de chacun des laboratoires, l'évaluation sera basée sur:
- les réponses aux questions du notebook;
- la justification et pertinence des algorithmes proposés et utilisés ;
- l'organisation de votre code source (SVP, pensez à commenter votre code source);


# Modules et bibliotèques python

### Import de bibliotèques

Ajouter une courte description aux bibliothèques que vous allez utiliser pour compléter ce notebook. N'hésitez pas à importer celles qui vous sont utiles.

In [ ]:
%pip install pandas seaborn matplotlib scikit-learn
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Partie 1 - Analyse exploratoire des données

On va commencer par regarder les données, c'est une pratique indispensable.

Pour ce lab, nous allons utiliser le dataset Bank Account Fraud (BAF). **La version à utiliser est celle du moodle.**



###  <font color=blue> Questions: </font>

1. Loader le dataset depuis le fichier csv (en utilisant la librairie [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) par exemple) et afficher le tableau de données. Si vous mettez vos données dans google colab, vous pouvez normalement y accéder au path: `/content/BAF_200k.csv`.

In [ ]:
dataset = df = pd.read_csv('BAF_200k.csv')
df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month
0,0,0.1,0.218119,110,7,20,0.015684,-1.013463,AD,687,...,28,0,500.0,0,INTERNET,15.611467,windows,1,1,7
1,0,0.1,0.373086,29,7,20,0.024277,19.342285,AA,1230,...,22,0,200.0,0,INTERNET,5.911984,other,0,1,0
2,0,0.2,0.515218,-1,64,20,0.015865,-1.333985,AB,2045,...,25,1,200.0,0,INTERNET,4.732999,other,1,1,4
3,0,0.6,0.899345,-1,56,20,0.018821,-1.524424,AC,1260,...,-1,0,200.0,0,INTERNET,8.210518,linux,1,1,3
4,0,0.9,0.794648,-1,64,50,0.027359,-1.254528,AB,1643,...,1,0,200.0,0,INTERNET,5.131610,other,0,1,0


2. Combien y a-t-il de catégories et quel est le nombre de données (nombre de transactions) dans ce dataset ?

In [22]:

# Code si besoin
print(f"nombre de colonnes {len(df.keys())}")
print(f"nombre de données {len(df.index)}")

nombre de colonnes 31
nombre de données 200000


Nombre de catégories: 31

Nombre de données: 200000

3. Afficher dans un graphe en baton (par exemple avec la librairie [matplotlib](https://matplotlib.org/) ou [seaborn](https://seaborn.pydata.org/)) le nombre de transactions étant des fraudes et celles n'en étant pas. Qu'est-ce que ce graphe vous permet d'observer ? Proposer une réponse courte.

In [ ]:
fraud_counts = df['fraud_bool'].value_counts()

fraud_counts.plot(kind='bar')
plt.title("Répartition des fraudes")
plt.xlabel("fraud_bool")
plt.ylabel("Nombre d'occurrences")
plt.xticks(ticks=[0, 1], labels=['False', 'True'], rotation=0)
plt.show()

Réponse ici:

4. Avant toute chose, nous allons séparer les données en trois ensembles : d'apprentissage (train_data), de validation (val_data) et de test (test_data). Nous ferons une **séparation stratifiée** car les données sont fortement débalancées entre fraudes ou non. Utiliser la fonction *train_test_split* de la librairie [scikit-learn train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pour faire cette séparation (On séparera les données en ensemble de 70%, 15%, 15%).

In [23]:
X = df.drop(columns=["fraud_bool"])
y = df["fraud_bool"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: (140000, 30), (140000,)
Validation: (30000, 30), (30000,)
Test: (30000, 30), (30000,)


5. S'assurer de l'intégrité des données en recherchant de potentielles valeurs manquantes. Afficher les catégories pour lesquelle il manque des données (celles-ci sont annotées par -1 dans le dataset), la quantitée de données manquantes par catégorie et le ratio de données manquantes par catégorie sur le total de données.

**Important**: Avant chaque étape de processing de vos données, il est important de toujours se rappeler que les données de validation et de test sont considérées "inconnues" préalablement à leur utilisation, et doivent représenter la distribution réelle des données. Il faut donc bien réfléchir quoi faire avec ces données pour ne pas biaiser vos résultats, voir les rendre faux!

In [ ]:
for column in df.columns:
    emptyCount = 0
    filtered_df = df[df[column] != -1]
    median = 0
    if(type(column[0]) == str):
        median = df[column].mode()[0]
    else:
        median = filtered_df[column].median()
        
    missingData = df[column].value_counts(-1, 0).count()
    ratio = missingData/len(df.index)
    print(f"Missing values for column {column}:{missingData}")
    print(f"Ratio of missing values:{ratio}")
    
    df[column].replace(-1, median)
    
        

Missing values for column fraud_bool:2
Ratio of missing values:1e-05
Missing values for column income:9
Ratio of missing values:4.5e-05
Missing values for column name_email_similarity:199947
Ratio of missing values:0.999735
Missing values for column prev_address_months_count:361
Ratio of missing values:0.001805
Missing values for column current_address_months_count:406
Ratio of missing values:0.00203
Missing values for column customer_age:9
Ratio of missing values:4.5e-05
Missing values for column days_since_request:199552
Ratio of missing values:0.99776
Missing values for column intended_balcon_amount:199768
Ratio of missing values:0.99884
Missing values for column payment_type:5
Ratio of missing values:2.5e-05
Missing values for column zip_count_4w:5855
Ratio of missing values:0.029275
Missing values for column velocity_6h:199939
Ratio of missing values:0.999695
Missing values for column velocity_24h:199973
Ratio of missing values:0.999865
Missing values for column velocity_4w:199934

6. Remplacer les valeurs manquantes par la valeur médiane de la colonne correspondante. Bien réfléchir à ce que cela implique pour les données de validation et de test.

In [ ]:
# Deja fait dans le bloc precedent

7. On notera que chacune des différentes catégories (colonnes) peut-être définie dans l'une des trois catégories suivantes:
- Catégorie 1: Données numériques (ex. montant de la transaction, etc.)
- Catégorie 2: Données catégorielles (ex. type de transaction, etc.)
- Catégorie 3: Données de type booléen (ex. transaction frauduleuse ou non, etc.)

Afficher une liste pour chaque catégorie de données (numérique *numeric_columns*, catégorielle *categorical_columns*, ou booléen *boolean_columns*), contenant le nom des colonnes (key) de données qui appartiennent à cette catégorie.

8. Pour les données de chaque catégorie numérique, afficher la distribution de données superposée entre les transations frauduleuses et celles n'en étant pas (pour les données d'entraînement). Vous pouvez de nouveau utiliser la librairie [matplotlib](https://matplotlib.org/) ou [seaborn](https://seaborn.pydata.org/) pour afficher les distributions de données.  

9. Considérant les distributions de données d'entrainement, donner une brève opinion / intuition sur la possibilité de classifier les transactions frauduleuses et celles n'en étant pas.

Réponse ici:

# Partie 2 - Préparation des données et classification par template matching

La partie 1 aura permis de faire une première analyse des données, et un premier prétraitement des données en s'attaquant aux données manquantes. Nous allons maintenant approfondir le traitement des données pour préparer la classification, puis appliquer la méthode appelée "template matching".

Un des points essentiels au bon fonctionnement des algorithmes d'apprentissage machine est, pour une grande majorité d'agorithmes, la normalization (scaling) des données. Cependant, le scaling des données peut-être fait de différentes manières, et peut aussi avoir un impact négatif sur les résultats de certains algorithmes si celle-ci n'est pas fait intelligemment. Vous êtes invités à survoler le papier suivant  - [paper link](https://www.sciencedirect.com/science/article/pii/S1568494622009735?fr=RR-2&ref=pdf_download&rr=93e5d0c1ecd57151) - pour motiver vos réponses aux questions à venir. N'hésitez pas à approfondir sa lecture au besoin.  

###  <font color=blue> Questions: </font>

1. Avant toute normalization de vos données, justifier pourquoi on ne pourra pas calculer la moyenne, l'écart-type, ou toute autre mesure statistique sur les bases val_data et test_data dans l'optique de leur normalization.

Réponse ici:

2. Commencer par choisir deux méthodes de normalisation différentes (ex. MinMaxScaler, StandardScaler, RobustScaler, etc.), en choisissant une normalization sensible aux outliers et une qui ne l'est pas. Appliquer chacune d'elles sur les données d'apprentissage. Normaliser aussi les données de validation et de test, tout en prenant en compte votre réponse à la question précédente.

Remarque: On nommera les nouvelles bases de train validation et test de la façons suivante: {data}_{scaler}. ex: train_data_minmax, val_data_minmax, test_data_minmax, train_data_standard, val_data_standard, test_data_standard.  

3. Nous allons maintenant classifier les opérations frauduleuses ou non en suivant une approche de *template matching* (plus proche prototype).

3a. Commencer par définir une fonction qui retourne un template pour chaque classe (fraude ou non). Dans le cas présent, chaque template est défini comme étant la moyenne des valeurs par colonne.

Plusieurs considérations sont importantes:

-> On pourra prendre la moyenne des données numériques et binaires (boolean)

-> Pour les données catégorielles, on pourra utiliser *pd.get_dummies* dans un premier temps (cette fonction créer des colonnes binaires pour chaque catégorie de la colonne - One-hot encoding). Ensuite, obtenir la moyenne de chaque colonne binaire.



3b. En utilisant ces templates, proposer une fonction permettant de classifier les données d'un ensemble (validation/test) étant donné les templates produits issus de la fonction précédente. La fonction devra retourner un numpy array pour les données prédites et les données réelles.

3c. Sur la base des fonctions précédentes, proposer une fonction d'évaluation qui appelle les fonction précédentes et retourne l'accuracy, une matrice de confusion, et un rapport de classification.  

4. Appliquer votre fonction d'évaluation pour les données de validation et de test pour vos trois approches (pas de normalisation, et vos deux types de normalisation).

5. Que pensez-vous de la mesure d'accuracy, est-ce une bonne mesure de performance pour ce problème ? Pourquoi ?

Réponse ici:

6. D'après la matrice de confusion et le rapport de classification, que pensez-vous de l'approche de template matching ?

Réponse ici:

7. Pouvez-vous déjà tirer des conclusions sur l'importance de la normalisation via les résultats obtenus ?

Réponse ici:

#### Résultats finaux

Afficher les f1-score obtenus par classe dans les tableaux suivants pour le template matching avec ou sans normalisation (%). On utilisera les meilleurs résultats obtenus entre les deux normalisations.

# Fraude
| Ensemble | Sans normalisation | Avec normalisation (*nom de la normalisation ici*) |                                 
:-|:-:|-:
| Train    |  XX,XX%       |   XX,XX%      |                   
| Val      |  XX,XX%       |   XX,XX%      |                             
| Test     |  XX,XX%       |   XX,XX%      |       


# Pas de fraude
| Ensemble | Sans normalisation | Avec normalisation (*nom de la normalisation ici*) |                                 
:-|:-:|-:
| Train    |  XX,XX%       |   XX,XX%      |                   
| Val      |  XX,XX%       |   XX,XX%      |                             
| Test     |  XX,XX%       |   XX,XX%      |   


# Fin